In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')

import numpy as np
from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from torch import optim, nn

from plotting import plot_model_outputs

import beer

output_notebook()

In [ ]:
mean = np.array([3., 2.])
cov = np.array([[2., 0.], [0., .2]])
data = np.random.multivariate_normal(mean, cov, size=100)
data[:, 1] = data[:, 1] + (data[:, 0]-mean[0])**2

fig = figure(
    title='Non-Linear subspace',
    width=400,
    height=400,
)
fig.circle(data[:, 0], data[:, 1])
x = np.linspace(-1, 7, 1000)
fig.line(x, (x-3)**2+2, color='red')
show(fig)

In [ ]:
obs_dim = 2
latent_dim = 2
nb_samples = 10

enc_struct = nn.Sequential(
    nn.Linear(obs_dim, 40),
    nn.ReLU(),
    nn.Linear(40, 40),
    nn.ReLU()
)
encoder = beer.models.GaussianMLP(enc_struct, 40, latent_dim)

dec_struct = nn.Sequential(
    nn.Linear(latent_dim, 40),
    nn.ReLU(),
    nn.Linear(40, 40),
    nn.ReLU()
)
decoder = beer.models.GaussianMLP(dec_struct, 40, obs_dim)

#latent_model = beer.models.NaturalIsotropicGaussian(latent_dim)
#latent_model = beer.models.NormalDiagonalCovariance.create(
#    latent_dim,
#    mean_prec=1e1,
#    prec_shape=1,
#    prec_rate=1.
#)

model = beer.models.VAE(encoder, decoder, latent_model, nb_samples)

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-6)
history = beer.inference.History(report_interval=250)

In [ ]:
beer.inference.run_training(data, model, optimizer, 2000, history, batch_size=20, lrate_latent_model=0.)

In [ ]:
plot_model_outputs(model, data)

In [ ]:
history.plot()